In this notebook we will be attempting to reshape the problem a bit. Instead of calculating air quality on an hourly level, we're instead going to use more traditional machine learning models in an attempt to classify if a future time period has poor air quality. The idea here is that while it may be hard to calculate exact air quality levels when there are so many outliers, we may be able to instead simply tell residents when we think conditions will be bad.

In [370]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from imblearn.under_sampling import RandomUnderSampler

In [371]:
aqi_df = pd.read_csv("../data/sh_bell/combined_meteor_and_sample.csv", index_col='Unnamed: 0', parse_dates=True)

In [372]:
aqi_df.drop(columns = ['AWD_cos', 'AWD_sin'], inplace=True)

In [373]:
aqi_df.head()

,AWS Mph WVc,Gust Mph Max,AvgT Deg_F Avg,ABP InHg Avg,Sample Value
2017-04-01 00:00:00,5.989,15.55,38.74,29.41,4.0
2017-04-01 01:00:00,5.059,11.30,39.46,29.44,6.0
2017-04-01 02:00:00,5.390,11.30,39.08,29.45,7.0
2017-04-01 03:00:00,5.233,11.59,38.95,29.47,6.0
2017-04-01 04:00:00,3.614,8.68,38.86,29.49,8.0


In [374]:
def df_shift(df, n_lags=1, n_obs=1):
    """
    Returns a dataframe with n_lags for all variables and n_obs for the sample value
    """
    n_vars = df.shape[1]
    cols, col_names = [], []
    # lags
    for i in range(n_lags, 0, -1):
        cols.append(df.shift(i))
        col_names += [f'{col} (t-{i})' for col in df.columns]
    # observations
    for i in range(0, n_obs):
        cols.append(df['Sample Value'].shift(-i))
        if i == 0:
            col_names.append('Sample Value (t)')
        else:
            col_names.append(f'Sample Value (t+{i})')
            
    shifted_df = pd.concat(cols, axis=1)
    shifted_df.columns = col_names
    shifted_df.dropna(inplace=True)
    
    return shifted_df

With this function, we can generate dataframes that have features from t - n_lags to t - 1, and observations from t to t + n_obs.

In [389]:
shifted_aqi_df = df_shift(aqi_df.copy(), 6, 1)
shifted_aqi_df.head()

,AWS Mph WVc (t-6),Gust Mph Max (t-6),AvgT Deg_F Avg (t-6),ABP InHg Avg (t-6),Sample Value (t-6),AWS Mph WVc (t-5),Gust Mph Max (t-5),AvgT Deg_F Avg (t-5),ABP InHg Avg (t-5),Sample Value (t-5),...,Gust Mph Max (t-2),AvgT Deg_F Avg (t-2),ABP InHg Avg (t-2),Sample Value (t-2),AWS Mph WVc (t-1),Gust Mph Max (t-1),AvgT Deg_F Avg (t-1),ABP InHg Avg (t-1),Sample Value (t-1),Sample Value (t)
2017-04-01 06:00:00,5.989,15.55,38.74,29.41,4.0,5.059,11.300,39.46,29.44,6.0,...,8.680,38.86,29.49,8.0,2.051,4.716,36.87,29.49,11.0,16.0
2017-04-01 07:00:00,5.059,11.30,39.46,29.44,6.0,5.390,11.300,39.08,29.45,7.0,...,4.716,36.87,29.49,11.0,2.134,4.190,35.21,29.51,16.0,20.0
2017-04-01 08:00:00,5.390,11.30,39.08,29.45,7.0,5.233,11.590,38.95,29.47,6.0,...,4.190,35.21,29.51,16.0,2.335,5.277,36.02,29.55,20.0,16.0
2017-04-01 09:00:00,5.233,11.59,38.95,29.47,6.0,3.614,8.680,38.86,29.49,8.0,...,5.277,36.02,29.55,20.0,3.432,9.590,40.24,29.57,16.0,7.0
2017-04-01 10:00:00,3.614,8.68,38.86,29.49,8.0,2.051,4.716,36.87,29.49,11.0,...,9.590,40.24,29.57,16.0,4.208,7.870,43.56,29.58,7.0,6.0


Now, we'll bin the sample values at times t based on their concentrations. There are no defined standards for 1 hour levels that I could find. And as discussed in the readme, the EPA is already rather conservative with the 24 hour levels that they have defined. Here, I'll be rather conservative and say that anything over 50 $\mu g / m^{3}$ should be considered harmful.

In [390]:
shifted_aqi_df['harmful'] = (shifted_aqi_df['Sample Value (t)'] > 50).astype(int)
shifted_aqi_df.drop('Sample Value (t)', axis=1, inplace=True)

In [391]:
shifted_aqi_df.reset_index(inplace=True)
shifted_aqi_df.drop('index', axis=1, inplace=True)

We can see here that there's a pretty large class imbalance. This will be addressed with random undersampling.

In [392]:
X = shifted_aqi_df[shifted_aqi_df.columns[:-1]]
y = shifted_aqi_df[shifted_aqi_df.columns[-1]]

In [393]:
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size = 0.25, random_state=42)
rus = RandomUnderSampler()
X_rus, y_rus = rus.fit_sample(X_train, y_train)

In [394]:
X_scaler = StandardScaler()
X_rus_scaled = X_scaler.fit_transform(X_rus)
X_test_scaled = X_scaler.transform(X_test)

In [395]:
model = RandomForestClassifier(n_estimators=100, bootstrap=True)
model.fit(X_rus_scaled, y_rus)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [396]:
y_pred = model.predict(X_test)

In [397]:
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix

roc_value = roc_auc_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)

In [398]:
fpr, tpr, thresh = roc_curve(y_test, y_pred)

In [399]:
cm

array([[4845,  247],
       [ 698,   53]], dtype=int64)

In [400]:
roc_value

0.5110325536101119

In [401]:
y_test

array([0, 0, 0, ..., 0, 0, 1])

In [402]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])